## The softmax function


The softmax function can be written as follows, where $i$ indexes an instance in a dataset and there are $K$ output classes. 

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k})}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})} $

In this notebook we will implement the softmax function.

### Step 1: Thinking through the dimensions of our data

In [2]:
import numpy as np

K = 3   # let K be the number of classes
J = 2   # let J be the number of features

W = np.random.rand(K, J)  

x_i = None

scores = None  # scores should be a 1D vector of raw scores, one for each class

### Step 2: Plotting the exp function

- Plot the exp function from -3 to 3. 

- What happens to the domain as the range gets very big or small?

In [3]:
import pandas as pd
import altair as alt

x = np.linspace(-3, 3, 50)
y = None

### Step 3: Naive softmax

In [22]:
def naive_sofmax(scores):
    '''
    Write a "naive" version of softmax that does not use vectorized operations
    This will be *way* slower than the numpy version, but that is OK
    We are just building intuition before jumping to the vectorized version
    When you don't know how to code something in ML, it can be good to start with a 
    simple version using loops before skipping ahead to the vectorized version
    '''
    numerator = None
    denominator = None

### Step 3: Vectorized code (aside)

Vectorized operations are much faster. A vectorized operation uses linear algebra packages like numpy to do computation very quickly using highly optimized code. Some implementations also implement vector operations in parallel.

Let's experiment with vectorized versions of a simple function summing a list of numbers, before moving on to a vectorized softmax.

In [5]:
import math 

def L2norm_no_vector(x):
    '''
    Your code here should not use vector operations
    
    input: a vector
    output, L2 norm of x, \sqrt{\Sigma x_i^2}
    '''
    return 42
        
def L2norm_with_vectors(x):
    '''
    Your code here should use vector operations

    input: a vector
    output, L2 norm of x, \sqrt{\Sigma x_i^2}
    '''
    return 42
        

Before going on, let's confirm that both implementations return the same outputs.

In [6]:
L2norm_no_vector([2,3,4]),  L2norm_with_vectors(np.asarray([2,3,4]))

(42, 42)

Let's test how long it takes our code to run as the list grows. 

In [7]:
from timeit import default_timer as timer

results = []

for n in range(1, 10000, 1000):
    runs = []
    for i in range(25): # take an average of 25 runs
        start = timer()
        L2norm_no_vector(np.random.rand(n))
        end = timer()
        runs.append(end - start)
    mean = np.mean(runs)
    results.append({"N": n, "f": 'L2norm_no_vector', "time": mean})
    
for n in range(1, 10000, 1000):
    runs = []
    for i in range(25): # take an average of 25 runs
        start = timer()
        L2norm_with_vectors(np.random.rand(n))
        end = timer()
        runs.append(end - start)
    mean = np.mean(runs)
    results.append({"N": n, "f": 'L2norm_with_vectors', "time": mean})
 
results = pd.DataFrame(results)
alt.Chart(results).mark_line().encode(
    x="N",
    y="time",
    color="f"
)

### Vectorized softmax

Now we are ready to take our first crack at a vectorized softmax function. Remember the equation for the softmax is as follows:

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k})}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})} $


In [8]:
def softmax(scores):
    '''
    input: 
        a scores function, of $K$ real values between -\inf and \inf
        
    output:
        a vector of probabilities that sums to 1
    '''
    return 42

softmax([1, 1])

42

#### Softmax questions

1. What happens when the scores are the same?
2. What happens when one score is bigger than the others
3. Why do you think this function might be called a "soft" max?
4. What happens if you pass in the scores `[10000, 1]`
4. What happens if you pass in the scores `[0, 1]`

### Numerically stable softmax 


$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k})}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})}$


There is some max score function $c$ in the vector of score functions. We can always divide the numerator and denominator of a fraction by $c$. It will affect the value of the numerator and denominator but not the ratio. 

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k}) / e^c}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}})/ e^c}$

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k}) exp(-c)}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}}) exp(-c)}$

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k} - c) }{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime} } -c ) }$

In [63]:
3/9

0.3333333333333333

In [64]:
(3/2)/(9/2)

0.3333333333333333

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k} - c)}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}}  - c)}$

In [67]:
def softmax_c(scores, c):
    '''
    input: 
        a scores function, of $K$ real values between -\inf and \inf
        
    output:
        a vector of probabilities that sums to 1
    '''
    return np.exp(scores - c)/np.sum(np.exp(scores - c))

softmax(np.asarray([1,2])),  softmax_c(np.asarray([1,2]), 2)

(array([0.26894142, 0.73105858]), array([0.26894142, 0.73105858]))

### Now let's take the log of the softmax

$p(y_i = k | x_i) = \frac{\text{exp}(\text{score}_{i, k} - c)}{\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}}  - c)}$

$ log p(y_i = k | x_i) = log (\text{score}_{i, k}) - c  - log {\sum_{k'}^K \text{exp}(\text{score}_{i,{k^\prime}}  - c)}$

We can use any $c$ we want so let's set it to the largest item in our scores vector.

In [17]:
def log_softmax(scores):
    '''
    input: 
        a scores function, of $K$ real values between -\inf and \inf
        
    output:
        a vector of probabilities that sums to 1
    '''
    return 42


np.exp(log_softmax([10000, 1]))

1.739274941520501e+18